### [ 데이터 전용 Dataset/DataLoader ]
- pytorch에서 데이터 관리 및 유지보수를 위한 클래스 제공
- Dataset : 사용자 데이터에 맞게 커스텀 클래스 생성
- DataLoader : 배치 크기만큼 데이터를 추출해주는 역할

[1] 모듈 로딩 및 데이터 준비 <hr>

In [60]:
# ==============================================================================
# [1-1] 모듈 로딩
# ==============================================================================
import torch
from torch.utils.data import Dataset , DataLoader 
import pandas as pd

# ==============================================================================
# [1-2] 데이터 준비 및 확인
# ==============================================================================
dataDF1 = pd.read_csv("../DATA/study_score_multi.csv")
print(dataDF1.head(2))

dataDF2 = pd.read_csv("../Data copy/iris.csv")
print(dataDF2.head(2))

   study_hours  sleep_hours  participation  score
0         1.45         5.28           97.4   32.8
1         2.17         5.46           22.4   32.1
   sepal.length  sepal.width  petal.length  petal.width variety
0           5.1          3.5           1.4          0.2  Setosa
1           4.9          3.0           1.4          0.2  Setosa


[2] 커스텀 데이터셋 클래스 생성 및 데이터 적용 <hr>

In [61]:
# ==============================================================================
# [2-1] 피쳐와 타겟 분리
# ==============================================================================
# 회귀용으로 타겟은 2D(Loss Function에 따라 변경 가능함)
featureDF = dataDF1[dataDF1.columns[:-1]]
targetDF = dataDF1[dataDF1.columns[-1:]]

print(f"featureDF : {featureDF.shape}, targetDF : {targetDF.shape}")

# ==============================================================================
# [2-2] 커스텀 데이터셋 클래스 정의
# ==============================================================================
# 클래스 이름 : RegDataset
# 부모 클래스 : Dataset
# 오버라이딩  : __init__(self)           : [필수] 피쳐, 타겟 / [선택] 행수, 컬럼수, 타겟의 수...
#           : __len__(self)            : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
#           : __getitem__(self, idx)   : 인스턴스명[idx] 시 자동 호출,
#                                        idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
# ==============================================================================
class RegDataset(Dataset):  # pytorch의 Dataset 클래스를 상속받아 회귀용 커스텀 데이터셋 클래스 정의

    # 피쳐와 타겟 저장 및 기타 속성 초기화
    def __init__(self, featureDF, targetDF):
        super().__init__()  # Dataset의 생성자를 호출해 초기화를 수행

        # 피쳐, 타겟 초기화 필수
        self.x = featureDF.values
        self.y = targetDF.values

        # 선택
        self.rows = featureDF.shape[0]
        self.cols = featureDF.shape[1]
        self.colnames = featureDF.columns

    # 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.rows    # 데이터셋의 전체 샘플 수 반환, len(dataset) 호출 시 자동 실행
    
    # 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[idx]에 자동호출됨
    def __getitem__(self, idx):
        xTS = torch.tensor(self.x[idx], dtype=torch.float32)
        yTS = torch.tensor(self.y[idx], dtype=torch.float32)
        return xTS, yTS
    
    # 커스텀 메서드
    def printInfo(self):
        print(f"컬럼명       : {self.colnames}")
        print(f"샘플수       : {self.rows}")
        print(f"피쳐수       : {self.cols}")
        print(f"데이터셋 형태 : 피쳐 {self.x.shape}, 타겟 {self.y.shape}")

featureDF : (5000, 3), targetDF : (5000, 1)


In [62]:
# ==============================================================================
# [2-3] 커스텀 데이터셋 인스턴스 생성 및 사용
# ==============================================================================
myDS = RegDataset(featureDF, targetDF)

print(f"타 입  : {type(myDS)}")
# print(f"컬럼명 : {myDS.colnames}")
# print(f"샘플수 : {myDS.rows}")
# print(f"타 겟 : {myDS.y.shape}")

타 입  : <class '__main__.RegDataset'>


In [63]:
# 메서드 사용
myDS.printInfo()

컬럼명       : Index(['study_hours', 'sleep_hours', 'participation'], dtype='object')
샘플수       : 5000
피쳐수       : 3
데이터셋 형태 : 피쳐 (5000, 3), 타겟 (5000, 1)


In [ ]:
# 필수 오버라이딩 메서드
print(len(myDS))
print(myDS[0])  # 0번째 텐서화된 피쳐, 타겟 형태
print(myDS[1])

5000
(tensor([ 1.4500,  5.2800, 97.4000]), tensor([32.8000]))
(tensor([ 2.1700,  5.4600, 22.4000]), tensor([32.1000]))


In [ ]:
# ==============================================================================
## [2-4] DataLoader를 활용한 데이터 추출 => 배치크기만큼 데이터 추출
# ==============================================================================
BATCH_SIZE = 53     # 한 번에 학습할 데이터의 묶음 크기
                    # 모델이 문제를 풀 때 53문제씩 끊어서 푼다는 의미

# DataLoader: myDS 데이터셋을 가져와서 규칙대로 데이터를 공급                    
myDL = DataLoader(myDS, 
                  batch_size=BATCH_SIZE, 
                  shuffle=True,     # 데이터 무작위로 꺼냄
                  drop_last=True)   # 53개씩 묶으면 자투리 데이터 발생 -> 버림

print(f"타입 : {type(myDL)}")

타입 : <class 'torch.utils.data.dataloader.DataLoader'>


In [66]:
# 배치 크기만큼 데이터를 추출 ======================================================
# DataLoader            : 추출 데이터 인덱스
# Dataset.__getitem__() : 인덱스에 해당하는 피쳐와 타겟 텐서
# ==============================================================================
for xTS, yTS in myDL:
    print(xTS.shape, yTS.shape)

torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
torch.Size([53, 3]) torch.Size([53, 1])
